# 모델 저장과 복원

## 설정
### 1. 설치와 임포트

In [2]:
!pip install -q h5py pyyaml

In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

import os

import tensorflow as tf
from tensorflow import keras

tf.__version__

'2.2.0-dev20200303'

In [4]:
#예제 데이터셋 받기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### 2. 모델 정의

In [5]:
# 간단한 Sequential 모델을 반환합니다
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model


# 모델 객체를 만듭니다
model = create_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### 3. 훈련하는 동안 체크포인트 저장하기
- 훈련 중간과 마지막에 체크포인트를 자동으로 저장하도록 해 다시 훈련하지 않고 모델을 재사용하거나 훈련 과정이 중지된 경우 이어서 훈련을 진행할 수 있음
- `tf.keras.callbacks.ModelCheckpoint` callback method를 통해 여러가지 매개변수를 제공받음

#### 체크포인트 콜백 사용하기

In [6]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 체크포인트 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model = create_model()

model.fit(train_images, train_labels,  epochs = 10,
          validation_data = (test_images,test_labels),
          callbacks = [cp_callback])  # 훈련 단계에 콜백을 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
23/32 [====================>.........] - ETA: 0s - loss: 1.3260 - accuracy: 0.6101
Epoch 00001: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 9ms/step - loss: 1.1705 - accuracy: 0.6430 - val_loss: 0.7452 - val_accuracy: 0.7860
Epoch 2/10
21/32 [==================>...........] - ETA: 0s - loss: 0.4298 - accuracy: 0.8795
Epoch 00002: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 5ms/step - loss: 0.4175 - accuracy: 0.8840 - val_loss: 0.5431 - val_accuracy: 0.8400
Epoch 3/10
29/32 [==========================>...] - ETA: 0s - loss: 0.2929 - accuracy: 0.9235
Epoch 00003: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 4ms/step - loss: 0.2957 - accuracy: 0.9180 - val_loss: 0.5032 - val_accuracy: 0.8430
Epoch 4/10
30/32 [===========================>..] - ETA: 0s - loss: 0.2286 - accuracy: 0.9438
Epoch 00004: saving model to training_1/cp.ckpt
32/32 [==============================] - 0s 4

텐서플로 체크포인트 파일을 만들고 에포크가 종료될 때마다 업데이트


In [7]:
!ls {checkpoint_dir}

checkpoint                  cp.ckpt.index
cp.ckpt.data-00000-of-00001


- 훈련하지 않은 새로운 모델을 만들어 볼 것임.
- 가중치만 복원할 땐 원본 모델과 동일한 구조로 모델을 만들어야 함.
- 다른 <u>객체</u>이지만 동일한 구조로 모델을 만들었으므로 가중치를 공유할 수 있음

In [8]:
model = create_model()

loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4003 - accuracy: 0.0960
훈련되지 않은 모델의 정확도:  9.60%


- 체크포인트에서 가중치를 로드하고 다시 평가

In [9]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4137 - accuracy: 0.8690
복원된 모델의 정확도: 86.90%


#### 체크포인트 콜백 매개변수
- 체크포인트 이름을 고유하게 만들거나 체크포인트 주기를 조정할 수 있음
- 새로운 모델을 훈련하고 다섯번의 에포크마다 고유한 이름으로 체크포인트 저장

In [10]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    checkpoint_path, verbose=1, save_weights_only=True,
    # 다섯 번째 에포크마다 가중치를 저장합니다
    period=5)

model = create_model()
model.save_weights(checkpoint_path.format(epoch=0))
model.fit(train_images, train_labels,
          epochs = 50, callbacks = [cp_callback],
          validation_data = (test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [11]:
! ls {checkpoint_dir}

checkpoint                       cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001 cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index               cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001 cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index               cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001 cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index               cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001 cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index               cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001 cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index               cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [12]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

- 모델 초기화하고 최근 체크포인트를 로드하여 테스트

In [13]:
model = create_model()
model.load_weights(latest)
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4916 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


### 4. 체크포인트 파일
- 체크포인트가 담고 있는 것
  - 모델의 가중치를 포함하는 하나 이상의 샤드(shard)
  - 가중치가 어느 샤드에 저장되어 있는지를 나타내는 인덱스 파일


### 5. 수동으로 가중치 저장하기
 - `Model.save_weights`를 사용하여 수동으로 가중치 저장

In [14]:
# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 가중치를 복원합니다
model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4916 - accuracy: 0.8780
복원된 모델의 정확도: 87.80%


### 6. 모델 전체를 저장하기
- 전체 모델을 하나의 파일에 저장.
- 가중치, 모델 구성, 옵티마이저에 지정한 설정끼지 포함
- 두 가지 방법: <b>`HDF5`</b>, <b>`Saved Model`</b>

#### 1) HDF5 파일로 저장하기
- 저장된 모델을 하나의 이진파일처럼 다룰 수 있음

In [15]:
model = create_model()

model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
model.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.2048 - accuracy: 0.6570
Epoch 2/5
32/32 [==============================] - 0s 1ms/step - loss: 0.4385 - accuracy: 0.8780
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2719 - accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2099 - accuracy: 0.9430
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1464 - accuracy: 0.9700


- <b>이 저장된 파일로 모델을 다시 만들어 보고 정확도 확인하기</b>

In [16]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = keras.models.load_model('my_model.h5')
new_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))


32/32 - 0s - loss: 0.4283 - accuracy: 0.8630
복원된 모델의 정확도: 86.30%


#### 2) saved_model 사용하기
- model을 serialize할 때(저장할 때) 쓰이는 방법
- `tf.keras.models.load_model`을 사용해 저장된 model을 불러올 수 있음

In [18]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

Epoch 1/5
32/32 [==============================] - 0s 2ms/step - loss: 1.1344 - accuracy: 0.6820
Epoch 2/5
32/32 [==============================] - 0s 2ms/step - loss: 0.4242 - accuracy: 0.8820
Epoch 3/5
32/32 [==============================] - 0s 2ms/step - loss: 0.2880 - accuracy: 0.9220
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2203 - accuracy: 0.9580
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1472 - accuracy: 0.9690
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [19]:
# 디렉토리 확인
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

my_model
assets         saved_model.pb variables


- 저장된 모델로부터 Keras model 로드

In [20]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_7 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [21]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100*acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4466 - accuracy: 0.8520
Restored model, accuracy: 85.20%
(1000, 10)
